[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PetiteIA/schema_mechanism/blob/master/experiments/small_loop.ipynb)

# THE SMALL LOOP ENVIRONMENT

In [89]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from ipywidgets import Button, HBox,VBox, Output
from IPython.display import display

## Define the actions

In [90]:
FORWARD = 0
TURN_LEFT = 1
TURN_RIGHT = 2
FEEL_FRONT = 3
FEEL_LEFT = 4
FEEL_RIGHT = 5

## Define the environment 

Choose the palette with the help of https://coolors.co/palettes/trending

In [91]:
colors = ['white', 'green', 'yellow', 'yellow', 'red']
colors = ["#BE9CC7", '#9F8DC3', '#E3BAD5', '#5C5792', "red"]
agent_color = "#211A44"
colors = ["#eec9c9", '#e8b6b6', '#f0f1f3', '#8b94a3', "red"]
agent_color = "#727c8f"
colors = ["#cccccc", '#a5a5a5', '#f2f2f2', '#7f7f7f', "red"]
agent_color = "#4392F1"
size = 200

empty_color = '#b0b0b0'
wall_color = '#b0b0b0'  # '#5C946E'
feel_empty_color = '#ffffff'
feel_wall_color = '#535865'
bump_color = "#F93943"
agent_color = "#1976D2"
colors = [empty_color, wall_color, feel_empty_color, feel_wall_color, bump_color]
#colors = ["#D6D6D6", '#5C946E', '#FAE2DB', '#535865', "#F93943"]
colors = ["#c0c0c0", '#b0b0b0', '#ffffff', '#535865', "#F93943"]

In [94]:
# Directions
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
# Display codes
FEELING_EMPTY = 2
FEELING_WALL = 3
BUMPING = 4

In [116]:
class SmallLoop():
    def __init__(self, position, direction):
        self.grid = np.array([
            [1, 1, 1, 1, 1, 1], 
            [1, 0, 0, 0, 1, 1],
            [1, 0, 1, 0, 0, 1],
            [1, 0, 1, 1, 0, 1],
            [1, 0, 0, 0, 0, 1],
            [1, 1, 1, 1, 1, 1]
        ])
        self.maze = self.grid.copy()
        self.position = np.array(position)  # Using NumPy array of shape (2)
        self.direction = direction
        self.cmap = ListedColormap(colors)
        self.norm = BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5, 4.5], self.cmap.N)
        self.marker_size = 400
        self.marker_map = {LEFT: '<', DOWN: 'v', RIGHT: '>', UP: '^'}
        self.marker_color = agent_color
        self.directions = np.array([
            [0, -1],  # Left
            [1, 0],   # Down
            [0, 1],   # Right
            [-1, 0]   # Up
            ])

    def outcome(self, action):
        """Update the grid. Return the outcome of the action."""
        result = 0
        x, y = self.position

        if action == FORWARD:  
            target_position = self.position + self.directions[self.direction]
            if self.grid[tuple(target_position)] == 0:
                self.position[:] = target_position
            else:
                result = 1
                self.maze[tuple(target_position)] = BUMPING
        
        elif action == TURN_RIGHT:
            self.direction = {LEFT: UP, DOWN: LEFT, RIGHT: DOWN, UP: RIGHT}[self.direction]
        
        elif action == TURN_LEFT:
            self.direction = {LEFT: DOWN, DOWN: RIGHT, RIGHT: UP, UP: LEFT}[self.direction]
        
        elif action == FEEL_FRONT:
            feeling_position = self.position + self.directions[self.direction]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = 1
                self.maze[tuple(feeling_position)] = FEELING_WALL
        
        elif action == FEEL_LEFT:
            feeling_position = self.position + self.directions[(self.direction + 1) % 4]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = 1
                self.maze[tuple(feeling_position)] = FEELING_WALL
        
        elif action == FEEL_RIGHT:
            feeling_position = self.position + self.directions[self.direction - 1]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = 1
                self.maze[tuple(feeling_position)] = FEELING_WALL

        print(f"Line: {self.position[0]}, Column: {self.position[1]}, direction: {self.direction}")
        return result  
    
    def display(self):
        """Display the grid in the notebook"""
        out.clear_output(wait=True)
        with out:
            fig, ax = plt.subplots()
            ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
            plt.scatter(self.position[1], self.position[0], s=self.marker_size, marker=self.marker_map[self.direction], c=self.marker_color)
            plt.show()
    
    def save(self, step):
        """Save the display as a PNG file"""
        fig, ax = plt.subplots()
        ax.set_xticks([])
        ax.set_yticks([])
        ax.axis('off')
        ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
        plt.scatter(self.position[1], self.position[0], s=self.marker_size, marker=self.marker_map[self.direction], c=self.marker_color)
        ax.text(4.5, 0, f"{step:>3}", fontsize=12, color='White')
        plt.savefig(f"{save_dir}/{step:03}.png", bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close(fig)
    
    def clear(self, clear):
        """Clear the grid display"""
        if clear:
            self.maze[:, :] = self.grid


## Display the environment

In [117]:
small_loop = SmallLoop([1, 1], 0)

Create the control buttons

In [118]:
def forward(change):
    small_loop.outcome(FORWARD)
    small_loop.display()
    small_loop.clear(True)

def turn_left(change):
    small_loop.outcome(TURN_LEFT)
    small_loop.display()
    small_loop.clear(True)

def turn_right(change):
    small_loop.outcome(TURN_RIGHT)
    small_loop.display()
    small_loop.clear(True)

def feel_left(change):
    small_loop.outcome(FEEL_LEFT)
    small_loop.display()
    small_loop.clear(True)

def feel_right(change):
    small_loop.outcome(FEEL_RIGHT)
    small_loop.display()
    small_loop.clear(True)

def feel_front(change):
    small_loop.outcome(FEEL_FRONT)
    small_loop.display()
    small_loop.clear(True)

# Create Start and Stop buttons
forward_button = Button(description="Forward")
left_button = Button(description="Turn left")
right_button = Button(description="Turn right")
feel_left_button = Button(description="Feel left")
feel_front_button = Button(description="Feel Front")
feel_right_button = Button(description="Feel right")

# Bind button clicks to functions
forward_button.on_click(forward)
left_button.on_click(turn_left)
right_button.on_click(turn_right)
feel_left_button.on_click(feel_left)
feel_right_button.on_click(feel_right)
feel_front_button.on_click(feel_front)

# Create the control bar
buttons = HBox([forward_button, left_button, right_button, feel_front_button, feel_left_button, feel_right_button])

In [119]:
out = Output()
small_loop.display()
display(out)
display(buttons)

Output()

## Enact actions

Use `Ctrl+Enter` to run the cell below and stay on it.

In [11]:
result = small_loop.outcome(FORWARD) # 0
small_loop.display()
print(f"outcome: {result}") 

Line: 4, Column: 1, direction: 1
outcome: 0


In [109]:
result = small_loop.outcome(TURN_LEFT) # 1
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 1
outcome: 0


In [117]:
result = small_loop.outcome(TURN_RIGHT) # 2
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 3
outcome: 0


In [108]:
result = small_loop.outcome(FEEL_FRONT) # 3
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 1


In [105]:
result = small_loop.outcome(FEEL_LEFT) # 4
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 0


In [107]:
result = small_loop.outcome(FEEL_RIGHT) # 5
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 1
